In [1]:
include("slim_reader.jl")
include("Encoder.jl")
using Knet

┌ Warning: `getindex(o::PyObject, s::AbstractString)` is deprecated in favor of dot overloading (`getproperty`) so elements should now be accessed as e.g. `o."s"` instead of `o["s"]`.
│   caller = top-level scope at slim_reader.jl:6
└ @ Core /home/ec2-user/comp492/Text2Image-cemil-slim-dataset-reader/slim_reader.jl:6


In [2]:
BATCH_SIZE = 50
slim_data_dir = "/home/ec2-user/efs/slim-dataset"

"/home/ec2-user/efs/slim-dataset"

In [3]:
syn_train_files = Slim_Reader.get_train_filenames(slim_data_dir, true, false)
# Construct training dataset object for synthetic data
syn_train_data = Slim_Reader.make_dataset_object(syn_train_files, BATCH_SIZE)




Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_initializable_iterator(dataset)`.



2019-11-16 11:02:33.865132: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
2019-11-16 11:02:33.915734: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300080000 Hz
2019-11-16 11:02:33.916289: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x661e370 executing computations on platform Host. Devices:
2019-11-16 11:02:33.916317: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): <undefined>, <undefined>
OMP: Info #212: KMP_AFFINITY: decoding x2APIC ids.
OMP: Info #210: KMP_AFFINITY: Affinity capable, using global cpuid leaf 11 info
OMP: Info #154: KMP_AFFINITY: Initial OS proc set respected: 0-3
OMP: Info #156: KMP_AFFINITY: 4 available OS procs
OMP: Info #157: K

Dict{String,Any} with 4 entries:
  "sess"         => PyObject <tensorflow.python.client.session.Session object a…
  "filenames"    => PyObject <tf.Tensor 'Placeholder:0' shape=(?,) dtype=string>
  "iterator"     => PyObject <tensorflow.python.data.ops.iterator_ops.Iterator …
  "next_element" => (((PyObject <tf.Tensor 'IteratorGetNext:0' shape=(?, 9, ?, …

In [4]:
# Iterate through the dataset and get the next batch
syn_train_batch = Slim_Reader.get_next_batch(syn_train_data, KnetArray{Float32}, false, true);

In [5]:
aimages, acaptions, acameras = getdata(syn_train_batch)

(K32(50,10,128,128,3)[0.6039216⋯], ["There is a large yellow cylinder to the right of a pink ico. The cylinder is in front of the ico.  " "There is a large yellow cylinder to the left.  " … "There is a pink ico behind a large yellow cylinder. The ico is to the left of the cylinder.  " "There is a pink ico behind a large yellow cylinder. The ico is to the left of the cylinder.  "; "There is a large pink box far away.  " "There is a large pink box to the left of a purple torus.  " … "There is a large pink box far away.  " "There is a large pink box in front of a purple torus.  "; … ; "There is a large peach cylinder in front of a green cone. The cylinder is close to the cone. There is a pink capsule in front of a large peach cylinder. There is a pink capsule to the right of a green cone. The capsule is in front of the cone.  " "There is a green cone close to a large peach cylinder. The cone is in front of the cylinder. There is a pink capsule behind a large peach cylinder. The capsule is

In [6]:
# create a dictionary of known words
vocabid = Dict()
vocabid = Dict("" => 1) # both padding and whitespace KnetArray(zeros(EMBEDDING_SIZE))
vocab = reshape(KnetArray(zeros(EMBEDDING_SIZE)),(EMBEDDING_SIZE,1))

64×1 KnetArray{Float64,2}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 ⋮  
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [7]:
function getembed(vocabid, vocab, w)
    if haskey(vocabid, w)
        return vocab[:,vocabid[w]]
    else
        println("There is no such word in the vocab!!!!")
    end
end

getembed (generic function with 1 method)

In [8]:
function updatevocabs!(vocabid, vocab, w)
    if haskey(vocabid, w)
        println("This word already exists...")
        return vocabid, vocab
    else
        if size(vocab,2) == length(vocabid)
            println("Vocab and vocabid are in order!")
            vocabid[w] = length(vocabid)+1
            vocab = hcat(vocab, KnetArray(Knet.xavier(EMBEDDING_SIZE)))
            return vocabid, vocab
        else
            return nothing
        end
    end    
end

updatevocabs! (generic function with 1 method)

In [9]:
function arrange(batchofcaptions, vocabid, vocab)
    int_batchofcaptions = []
    for i in 1:BATCH_SIZE
        int_captions = []
        captions = batchofcaptions[i,:]
        for c in captions
            int_cap = []
            str = split(c, " ")
            for i in 1:MAX_LENGTH
               if i <= length(str)
                    vocabid, vocab = updatevocabs!(vocabid, vocab, str[i])
                    push!(int_cap, getembed(vocabid, vocab, str[i])) #KnetArray(Knet.xavier(EMBEDDING_SIZE))
                else
                    push!(int_cap, getembed(vocabid, vocab, "")) # padding, same embedding with whitespace
                end
            end
            push!(int_captions, hcat(int_cap...))
        end
        push!(int_batchofcaptions, int_captions)
    end
    return vocabid, vocab, int_batchofcaptions
end

arrange (generic function with 1 method)

In [10]:
vocabid, vocab, caption_embeds = arrange(acaptions, vocabid, vocab)

Vocab and vocabid are in order!
Vocab and vocabid are in order!
Vocab and vocabid are in order!
Vocab and vocabid are in order!
Vocab and vocabid are in order!
Vocab and vocabid are in order!
Vocab and vocabid are in order!
Vocab and vocabid are in order!
Vocab and vocabid are in order!
Vocab and vocabid are in order!
This word already exists...
Vocab and vocabid are in order!
Vocab and vocabid are in order!
Vocab and vocabid are in order!
This word already exists...
This word already exists...
Vocab and vocabid are in order!
Vocab and vocabid are in order!
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
Vocab and vocabid are in order!
This word already exists...
This word al

This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already ex

This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already ex

This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already ex

This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already ex

...

This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already ex

This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already ex

This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already ex

This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already exists...
This word already ex

(Dict("pink" => 12,"cone" => 40,"right" => 10,"capsule" => 29,"teal" => 47,"of" => 11,"purple" => 25,"cylinder" => 7,"right." => 44,"to" => 8…), K64(64,48)[0.0⋯], Any[Any[K64(64,60)[0.13779732636314437⋯], K64(64,60)[0.13779732636314437⋯], K64(64,60)[0.13779732636314437⋯], K64(64,60)[0.13779732636314437⋯], K64(64,60)[0.13779732636314437⋯], K64(64,60)[0.13779732636314437⋯], K64(64,60)[0.13779732636314437⋯], K64(64,60)[0.13779732636314437⋯], K64(64,60)[0.13779732636314437⋯], K64(64,60)[0.13779732636314437⋯]], Any[K64(64,60)[0.13779732636314437⋯], K64(64,60)[0.13779732636314437⋯], K64(64,60)[0.13779732636314437⋯], K64(64,60)[0.13779732636314437⋯], K64(64,60)[0.13779732636314437⋯], K64(64,60)[0.13779732636314437⋯], K64(64,60)[0.13779732636314437⋯], K64(64,60)[0.13779732636314437⋯], K64(64,60)[0.13779732636314437⋯], K64(64,60)[0.13779732636314437⋯]], Any[K64(64,60)[0.13779732636314437⋯], K64(64,60)[0.13779732636314437⋯], K64(64,60)[0.13779732636314437⋯], K64(64,60)[0.13779732636314437⋯], K64

In [11]:
caption_embeds[1]

10-element Array{Any,1}:
 K64(64,60)[0.13779732636314437⋯]
 K64(64,60)[0.13779732636314437⋯]
 K64(64,60)[0.13779732636314437⋯]
 K64(64,60)[0.13779732636314437⋯]
 K64(64,60)[0.13779732636314437⋯]
 K64(64,60)[0.13779732636314437⋯]
 K64(64,60)[0.13779732636314437⋯]
 K64(64,60)[0.13779732636314437⋯]
 K64(64,60)[0.13779732636314437⋯]
 K64(64,60)[0.13779732636314437⋯]

In [12]:
function createconvinput(caption_embeds)
    o = []
    for i in 1:BATCH_SIZE
        push!(o, cat(transpose(caption_embeds[i])...,dims=3))
    end
    return o = cat(o..., dims=3)
end

createconvinput (generic function with 1 method)

In [13]:
input = createconvinput(caption_embeds)

60×64×500 KnetArray{Float64,3}:
[:, :, 1] =
  0.137797    0.13096     -0.017507    …  -0.092361    -0.142903  
 -0.123112    0.0945619    0.156071       -0.0361477   -0.103255  
 -0.116633   -0.0749414   -0.0748445       0.0369269    0.00225376
 -0.118588    0.00238167  -0.102017        0.0946392   -0.0991475 
  0.127913    0.116823    -0.125899       -0.114881     0.153341  
  0.0732793  -0.117442     0.0972961   …   0.033963    -0.0103143 
  0.0669035   0.0074943    0.00588001      0.039106    -0.0967719 
 -0.159004    0.094124     0.0703406       0.0829859   -0.0367256 
  0.0288129   0.0701381    0.026573        0.152026    -0.132866  
 -0.0516615   0.0861344    0.138246        0.109351     0.0411477 
 -0.116633   -0.0749414   -0.0748445   …   0.0369269    0.00225376
  0.0481286  -0.00179858  -0.0551868       0.00602561   0.0550106 
 -0.0629376   0.134012     0.16817        -0.135037     0.127515  
  ⋮                                    ⋱                          
  0.0         0.0 

In [14]:
@doc conv4

```
conv4(w, x; kwargs...)
```

Execute convolutions or cross-correlations using filters specified with `w` over tensor `x`.

Currently KnetArray{Float32/64,4/5} and Array{Float32/64,4} are supported as `w` and `x`.  If `w` has dimensions `(W1,W2,...,I,O)` and `x` has dimensions `(X1,X2,...,I,N)`, the result `y` will have dimensions `(Y1,Y2,...,O,N)` where

```
Yi=1+floor((Xi+2*padding[i]-Wi)/stride[i])
```

Here `I` is the number of input channels, `O` is the number of output channels, `N` is the number of instances, and `Wi,Xi,Yi` are spatial dimensions.  `padding` and `stride` are keyword arguments that can be specified as a single number (in which case they apply to all dimensions), or an array/tuple with entries for each spatial dimension.

# Keywords

  * `padding=0`: the number of extra zeros implicitly concatenated at the start and at the end of each dimension.
  * `stride=1`: the number of elements to slide to reach the next filtering window.
  * `dilation=1`: dilation factor for each dimension.
  * `mode=0`: 0 for convolution and 1 for cross-correlation.
  * `alpha=1`: can be used to scale the result.
  * `handle`: handle to a previously created cuDNN context. Defaults to a Knet allocated handle.


In [15]:
filter = reshape(KnetArray(Knet.xavier(30*EMBEDDING_SIZE*32)), (1,30,EMBEDDING_SIZE,32)) # output size = embedding/input size
input = reshape(input, (1, MAX_LENGTH, EMBEDDING_SIZE, BATCH_SIZE*NUM_CAPTIONS_PER_SCENE))

1×60×64×500 KnetArray{Float64,4}:
[:, :, 1, 1] =
 0.137797  -0.123112  -0.116633  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0

[:, :, 2, 1] =
 0.13096  0.0945619  -0.0749414  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0

[:, :, 3, 1] =
 -0.017507  0.156071  -0.0748445  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0

...

[:, :, 62, 1] =
 -0.00663056  -0.0463905  0.00340827  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0

[:, :, 63, 1] =
 -0.092361  -0.0361477  0.0369269  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0

[:, :, 64, 1] =
 -0.142903  -0.103255  0.00225376  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0

[:, :, 1, 2] =
 0.137797  -0.123112  -0.116633  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0

[:, :, 2, 2] =
 0.13096  0.0945619  -0.0749414  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0

[:, :, 3, 2] =
 -0.017507  0.156071  -0.0748445  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0

...

[:, :, 62, 2] =
 -0.00663056  -0.0463905  0.00340827  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0

[:, :, 63, 2] =
 -0.092361  -0.0361477  0.0369269  …  0.0  0.0  0.0  0.0  0.

In [16]:
@show y = conv4(filter, input, dilation=2)

y = conv4(filter, input, dilation=2) = K64(1,2,32,500)[-0.0011552335169203328⋯]


1×2×32×500 KnetArray{Float64,4}:
[:, :, 1, 1] =
 -0.00115523  -0.0109114

[:, :, 2, 1] =
 -0.0087934  0.00706837

[:, :, 3, 1] =
 0.00683621  0.00667209

...

[:, :, 30, 1] =
 0.00972923  -0.0131102

[:, :, 31, 1] =
 -0.0108216  0.0028484

[:, :, 32, 1] =
 -0.010127  0.00761386

[:, :, 1, 2] =
 -0.0025868  -0.00338106

[:, :, 2, 2] =
 0.00388787  0.000602739

[:, :, 3, 2] =
 -0.000227692  -0.00111839

...

[:, :, 30, 2] =
 -0.000742195  0.00179599

[:, :, 31, 2] =
 0.00252244  0.000924172

[:, :, 32, 2] =
 -0.00659177  0.00890555

[:, :, 1, 3] =
 -0.00168525  -0.00359879

[:, :, 2, 3] =
 0.00531266  0.00206481

[:, :, 3, 3] =
 -0.00176955  -0.00586311

...

[:, :, 30, 3] =
 -0.0043706  0.00255134

[:, :, 31, 3] =
 0.00200177  0.000408439

[:, :, 32, 3] =
 -0.0034765  0.011975

...

[:, :, 1, 498] =
 0.00332734  -0.000691786

[:, :, 2, 498] =
 0.00237245  0.00792598

[:, :, 3, 498] =
 -0.000135456  0.00220531

...

[:, :, 30, 498] =
 -0.00987212  0.000295691

[:, :, 31, 498] =
 0.001843

In [17]:
function build_angles(tuplebatch)
    output = []
    for i in 1:BATCH_SIZE
        for j in 1:NUM_CAPTIONS_PER_SCENE
            sin_value = tuplebatch[i,j,3]/sqrt(tuplebatch[i,j,1]^2+tuplebatch[i,j,3]^2)
            cos_value = tuplebatch[i,j,1]/sqrt(tuplebatch[i,j,1]^2+tuplebatch[i,j,3]^2)
            push!(output, KnetArray([sin_value, cos_value])) 
        end
    end
    
    return cat(output..., dims=2)
end

build_angles (generic function with 1 method)

In [18]:
tuples = build_angles(acameras)

2×500 KnetArray{Float32,2}:
  0.923807  0.273762   0.424345  0.873171  …   0.293956  0.629109  0.948675
 -0.382859  0.961797  -0.9055    0.487415     -0.955819  0.777317  0.316251

In [19]:
struct Chain
    layers
    Chain(layers...) = new(layers)
end
(c::Chain)(x) = (for l in c.layers; x = l(x); end; x)

In [20]:
struct Layer0; w; b; end
Layer0(ir::Int, ic::Int, o::Int) = Layer0(param(o,ir),param0(o, ic))
(l::Layer0)(x) = (l.w * x .+ l.b)

In [21]:
mlp1=Chain(Layer0(2, 500, 32)) # MLP1 dimensionality 32

Chain((Layer0(P(KnetArray{Float32,2}(32,2)), P(KnetArray{Float32,2}(32,500))),))

In [22]:
mlp2=Chain(Layer0(96, 500, 128), Layer0(128, 500, 196), Layer0(196, 500, 256)) # MLP2 dimensionality 256

Chain((Layer0(P(KnetArray{Float32,2}(128,96)), P(KnetArray{Float32,2}(128,500))), Layer0(P(KnetArray{Float32,2}(196,128)), P(KnetArray{Float32,2}(196,500))), Layer0(P(KnetArray{Float32,2}(256,196)), P(KnetArray{Float32,2}(256,500)))))

In [23]:
cameras_hat = mlp1(tuples)

32×500 KnetArray{Float32,2}:
  0.0122048  -0.145353    0.106693   …   0.119428   -0.135503   -0.0857521
  0.112657    0.195242   -0.0580836     -0.0896932   0.232975    0.222478 
  0.0675194  -0.167816    0.158467       0.167166   -0.135344   -0.0545835
 -0.226873   -0.0429222  -0.120709      -0.0910462  -0.131031   -0.216942 
  0.0620509  -0.151055    0.143482       0.151169   -0.121323   -0.048072 
 -0.121299   -0.177462    0.0403105  …   0.0711656  -0.219222   -0.217932 
 -0.197392   -0.103904   -0.0598578     -0.0275902  -0.17826    -0.232664 
  0.205513    0.0935073   0.0722761      0.0401048   0.171431    0.232557 
  0.0752787   0.0275314   0.0310345      0.0199025   0.056307    0.0807513
 -0.0864067   0.172369   -0.174031      -0.181049    0.132281    0.0418848
  0.110754    0.0140809   0.0635904  …   0.049777    0.0573315   0.101371 
  0.0611089   0.270357   -0.143098      -0.176205    0.285134    0.229179 
 -0.0587842  -0.175505    0.0802693      0.103909   -0.192645   -0.1646

In [24]:
captions_hat = convert(KnetArray{Float32}, reshape(reshape(y, (2, 32, 500)), (64, 500)))
#cameras_hat = reshape(cameras_hat, (1, 32, 500))

64×500 KnetArray{Float32,2}:
 -0.00115523  -0.0025868    -0.00168525   …   0.00213294  -0.0116268 
 -0.0109114   -0.00338106   -0.00359879       0.00448285  -0.00385873
 -0.0087934    0.00388787    0.00531266      -0.0055775    0.00271827
  0.00706837   0.000602739   0.00206481       0.00900856   0.0143011 
  0.00683621  -0.000227692  -0.00176955       0.0309974   -0.00937796
  0.00667209  -0.00111839   -0.00586311   …   0.0111898    0.0134163 
 -0.0130604    0.00169059   -0.00368089      -0.00244722  -0.0128268 
 -0.00196996  -0.00135872   -0.000293457      0.0147609   -0.00273523
  0.00812651  -0.00250413   -0.00238972       0.00257699  -0.012693  
 -0.00512638   0.00128996    0.00468752      -0.0122057    0.00785021
  0.0060523    3.36649e-5    0.00218768   …  -0.0125132    0.0159293 
 -0.0116796   -0.00774759   -0.00930669      -0.00547444  -0.018441  
  0.00585321  -0.00119686    0.00377162       0.0260579    0.00232447
  ⋮                                       ⋱                  

In [25]:
h = mlp2(cat(captions_hat, cameras_hat, dims=1))

256×500 KnetArray{Float32,2}:
  0.00413893   0.0241342   -0.0131203    …   0.023831      0.0211363  
  0.00343304  -0.0105617    0.00943231      -0.0096858    -0.00498265 
 -0.0105258    0.0180212   -0.0189224        0.0132362     0.00445994 
 -0.0121183    0.00333506  -0.00824307      -0.00243126   -0.00755681 
  0.00230214   0.0143301   -0.00853422       0.0142452     0.0107962  
 -0.0232552   -0.00447389  -0.0118893    …  -0.012504     -0.0235201  
  0.019449    -0.0220081    0.0280765       -0.0132316     0.00118189 
  0.0123124   -0.00483809   0.0123671       -7.16424e-5    0.00793522 
 -0.0136876   -0.015106     0.00153316      -0.0208086    -0.0204312  
 -0.00291663  -0.00175739  -0.00139965      -0.00255985   -0.00223316 
 -0.00788122  -0.0214548    0.00995483   …  -0.0216311    -0.0209541  
 -0.0114414    0.00630085  -0.0125863        0.00266516   -0.00602575 
 -0.0101528   -0.0197829    0.00783734      -0.022552     -0.0203306  
  ⋮                                      ⋱     

In [26]:
function aggregate(h) # batch of h s
    r = []
    for i in 1:BATCH_SIZE
        push!(r, sum(h[:,10*(i-1)+1:10*i-1], dims=2)./(NUM_CAPTIONS_PER_SCENE-1)) # 1...9, 11...19, ...
    end
    return hcat(r...)
end

aggregate (generic function with 1 method)

In [27]:
r = aggregate(h)

256×50 KnetArray{Float32,2}:
  0.00101504   0.00212312    0.0146461    …   0.0120864     0.0122078  
  0.00277704   0.00207005   -0.00171196      -0.00189617   -0.00313737 
 -0.00734235  -0.00517691    0.000483017      0.00244243    0.0025999  
 -0.00612788  -0.00476191   -0.00763953      -0.00181978   -0.00455717 
  1.83414e-5   0.00046728    0.00786825       0.0048826     0.00619032 
 -0.0128793   -0.0115839    -0.0203128    …  -0.00999312   -0.0120587  
  0.0129311    0.0106231     0.00428625      -0.000696434  -0.000146382
  0.00790907   0.00728075    0.00751462       0.00325445    0.00367161 
 -0.00651677  -0.00651035   -0.0166111       -0.0107654    -0.0128674  
 -0.00158695  -0.0016165    -0.00249493      -0.00102753   -0.00106572 
 -0.00228571  -0.00339897   -0.0140902    …  -0.0104028    -0.0108588  
 -0.00736994  -0.00630533   -0.00625636      -0.00246575   -0.00160513 
 -0.00367064  -0.00374075   -0.0153384       -0.0102192    -0.0121537  
  ⋮                                

In [43]:
function createimgencinput(images) # images size: 50×10×128×128×3
    out = []
    for i in 1:BATCH_SIZE
        push!(out, permutedims(images[i,1:9,:,:,:], (2,3,4,1)))
    end
    return cat(out..., dims=4)
end

createimgencinput (generic function with 1 method)

In [45]:
imgencoderin = createimgencinput(aimages)

128×128×3×450 KnetArray{Float32,4}:
[:, :, 1, 1] =
 0.603922  0.603922  0.603922  0.603922  …  0.603922  0.603922  0.603922
 0.603922  0.603922  0.603922  0.603922     0.603922  0.603922  0.603922
 0.603922  0.603922  0.603922  0.603922     0.603922  0.603922  0.603922
 0.603922  0.603922  0.603922  0.603922     0.603922  0.603922  0.603922
 0.603922  0.603922  0.603922  0.603922     0.603922  0.603922  0.603922
 0.603922  0.603922  0.603922  0.603922  …  0.603922  0.603922  0.603922
 0.603922  0.603922  0.603922  0.603922     0.603922  0.603922  0.603922
 0.603922  0.603922  0.603922  0.603922     0.603922  0.603922  0.603922
 0.603922  0.603922  0.603922  0.603922     0.603922  0.603922  0.603922
 0.603922  0.603922  0.603922  0.603922     0.603922  0.603922  0.603922
 0.603922  0.603922  0.603922  0.603922  …  0.603922  0.603922  0.603922
 0.603922  0.603922  0.603922  0.603922     0.603922  0.603922  0.603922
 0.603922  0.603922  0.603922  0.603922     0.603922  0.603922  0.603922


In [94]:
imgencoderin2 = pool(imgencoderin, window=4, stride=4)

32×32×3×450 KnetArray{Float32,4}:
[:, :, 1, 1] =
 0.603922  0.603922  0.603922  0.603922  …  0.603922  0.603922  0.603922
 0.603922  0.603922  0.603922  0.603922     0.603922  0.603922  0.603922
 0.603922  0.603922  0.603922  0.603922     0.603922  0.603922  0.603922
 0.603922  0.603922  0.603922  0.603922     0.603922  0.603922  0.603922
 0.603922  0.603922  0.603922  0.603922     0.603922  0.603922  0.603922
 0.603922  0.603922  0.603922  0.603922  …  0.603922  0.603922  0.603922
 0.603922  0.603922  0.603922  0.603922     0.603922  0.603922  0.603922
 0.603922  0.603922  0.603922  0.603922     0.603922  0.603922  0.603922
 0.670588  0.678431  0.654902  0.639216     0.627451  0.67451   0.67451 
 0.564706  0.588235  0.623529  0.658824     0.658824  0.658824  0.439216
 0.596078  0.584314  0.588235  0.584314  …  0.411765  0.435294  0.427451
 0.607843  0.588235  0.592157  0.584314     0.411765  0.443137  0.435294
 0.572549  0.592157  0.596078  0.588235     0.411765  0.411765  0.427451
 ⋮

In [108]:
imgencoderin3 = reshape(imgencoderin2, (1,32,32,3,450))

1×32×32×3×450 KnetArray{Float32,5}:
[:, :, 1, 1, 1] =
 0.603922  0.603922  0.603922  0.603922  …  1.0  1.0  1.0  1.0  1.0  1.0  1.0

[:, :, 2, 1, 1] =
 0.603922  0.603922  0.603922  0.603922  …  1.0  1.0  1.0  1.0  1.0  1.0  1.0

[:, :, 3, 1, 1] =
 0.603922  0.603922  0.603922  0.603922  …  1.0  1.0  1.0  1.0  1.0  1.0  1.0

...

[:, :, 30, 1, 1] =
 0.603922  0.603922  0.603922  0.603922  …  0.894118  0.913725  0.933333

[:, :, 31, 1, 1] =
 0.603922  0.603922  0.603922  0.603922  …  0.878431  0.898039  0.921569

[:, :, 32, 1, 1] =
 0.603922  0.603922  0.603922  0.603922  …  0.909804  0.886275  0.909804

[:, :, 1, 2, 1] =
 0.886275  0.886275  0.886275  0.886275  …  1.0  1.0  1.0  1.0  1.0  1.0  1.0

[:, :, 2, 2, 1] =
 0.886275  0.886275  0.886275  0.886275  …  1.0  1.0  1.0  1.0  1.0  1.0  1.0

[:, :, 3, 2, 1] =
 0.886275  0.886275  0.886275  0.886275  …  1.0  1.0  1.0  1.0  1.0  1.0  1.0

...

[:, :, 30, 2, 1] =
 0.886275  0.886275  0.886275  0.886275  …  0.894118  0.913725  0.933333



In [159]:
filt = reshape(KnetArray{Float32}(Knet.xavier(32*32*3)), (32,32,3,1))

32×32×3×1 KnetArray{Float32,4}:
[:, :, 1, 1] =
  0.0159694    -0.0044134     0.0174023   …   0.0228288    -0.0229805  
  0.0104772     0.0076507    -0.0235611       0.00487948    0.0173796  
  0.0129107     0.0208258     0.0239657      -0.0112125     0.0211187  
 -0.00138419   -0.0024362     0.0241198      -0.00306026   -0.0236004  
 -0.00171284    0.0105941     0.0138889      -0.000175724   0.00801835 
  0.0151296     0.00598983    0.00770526  …  -0.0182945     0.0204351  
  0.0132481     0.0198748     0.00295993     -0.00672287    0.00644547 
  0.0143376    -0.00590555   -0.0207746      -0.0196666     0.022718   
  0.010365      0.0223988     0.00912147     -0.00378355   -0.000164603
  0.00764152   -0.0104908    -0.0118109       0.0135539     0.0248861  
  0.00688294    0.0226229     0.0196569   …  -0.00492487    0.0121016  
 -0.00520392    0.0199448    -0.0180849      -0.00783922    0.00532665 
  0.000853779   0.000337943  -0.0106961      -0.00144648   -0.0081135  
  ⋮              

In [160]:
conv4(imgencoderin2, filt) # O, N yerine N, O geliyor

1×1×450×1 KnetArray{Float32,4}:
[:, :, 1, 1] =
 0.3392464

[:, :, 2, 1] =
 0.069831826

[:, :, 3, 1] =
 0.43243355

...

[:, :, 448, 1] =
 0.15342474

[:, :, 449, 1] =
 0.18894443

[:, :, 450, 1] =
 0.11824841